In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
from ilqr.cost import QRCost, FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']
Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']


In [5]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt, final_state)

In [6]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'inverted_double_pendulum.xml')
dynamics = MujocoDynamics(xml_path, frame_skip = 4, use_multiprocessing = True)
print(dynamics.dt)

Finished loading process Finished loading process4804Finished loading process Finished loading processFinished loading process 
  48034807

Finished loading process4806
 4812Finished loading process4805

 4817
Finished loading process 4834
0.04
Finished loading process Finished loading process 48704857
Finished loading process
 4877
Finished loading process 4878


In [7]:
print(dynamics.state_size)
x_goal = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Instantenous state cost.
Q = np.eye(dynamics.state_size)
Q[0, 0] = 5.0
Q[1, 1] = 50.0
Q[2, 2] = 50.0
Q[3, 3] = 20.0
Q[4, 4] = 700.0
Q[5, 5] = 700.0



# Terminal state cost.
Q_terminal = 10 * Q

# Instantaneous control cost.
R = np.eye(1)

cost1 = QRCost(Q, R, Q_terminal=Q_terminal, x_goal=x_goal)

6


In [8]:
def l(x, u, i):
    c0 = x[0] ** 2
    c1 = 10 * ((np.cos(x[1]) - 1) ** 2)
    c2 = 10 * ((np.cos(x[2]) - 1) ** 2)
    c3 = x[3] ** 2
    c4 = x[4] ** 2
    c5 = x[5] ** 2
    cu = 0.5 * u[0] ** 2
    return c0 + c1 + c2 + c3 + c4 + c5 + cu

cost2 = FiniteDiffCost(l, lambda x, i: l(x, [0.0], i), 6, 1, use_multiprocessing = True)

Finished loading process Finished loading process4914Finished loading process  Finished loading process49224917Finished loading process
 

4938 4931Finished loading process

Finished loading process 4945 
4956
Finished loading process 4967
Finished loading process 4980
Finished loading process 4989Finished loading process
 4998Finished loading process
 5005


In [9]:
N = 100
x0 = np.array([0.0, np.random.uniform(-np.pi, np.pi), np.random.uniform(-np.pi, np.pi), 0.0, 0.0, 0.0])
#x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
#us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N)
mpc = RecedingHorizonController(x0, ilqr)

In [10]:
t0 = time.time()
J_hist = []
controls = mpc.control(us_init, step_size = 3, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []
for i in range(30):
    print('ITERATION', i, '\n')
    if i == 29:
        us.append(next(controls)[2])
    else:
        us.append(next(controls)[1])
    
print('time', time.time() - t0)
us = np.concatenate(us)

ITERATION 0 

iteration 0 accepted 17545.785008734456 [-2.15344449  8.75981493 -4.94454838 -3.57234412 -3.51931418 15.56698104]
iteration 1 accepted 16633.37560804239 [-2.33126068  9.52947735 -6.63525534 -3.89645697 -9.02680658 23.98386265]
iteration 2 accepted 14031.438309425337 [-2.97266234 10.06571703 -9.77827006 -0.26624165  9.79967067 -6.92501241]
iteration 3 accepted 13763.443930618952 [-2.97584379 10.04052483 -9.77548313 -0.37735981  9.52727833 -6.74719145]
iteration 4 accepted 13504.475910927293 [-2.97546758 10.01433899 -9.77651482 -0.38140733  9.2040402  -6.58934451]
iteration 5 accepted 13486.244983299865 [ -3.00237547   9.24348839 -10.72758357   0.26071155   6.82036681
 -11.28925088]
iteration 6 accepted 12814.138844688907 [-2.99964744 10.16203909 -8.78122226  0.26479362  7.97093486  4.52161862]
iteration 7 accepted 12436.357353591904 [-3.00155199 10.02655115 -8.72607994  0.27458511  8.43128886  4.21121927]
iteration 8 accepted 11668.282184382624 [-2.95969487  9.80934745 -8.

iteration 72 accepted 8285.299419787563 [-1.10925984  7.911671   -6.75849211 -1.46197568  2.87324601  0.16930291]
iteration 73 accepted 8277.041509720983 [-1.1215951   7.91071282 -6.7532982  -1.44310691  2.86321844  0.16058109]
iteration 74 accepted 8276.440522880901 [-1.09748782  7.89388954 -6.74717881 -1.41312266  2.87845867  0.11788774]
iteration 75 accepted 8273.131593107213 [-1.09761865  7.8898415  -6.74846082 -1.41367667  2.87317737  0.10722922]
iteration 76 accepted 8272.458171833146 [-1.15493651  7.89700463 -6.73321095 -1.33862598  2.86006066  0.08373018]
iteration 77 accepted 8272.36961781804 [-1.14626738e+00  7.86194718e+00 -6.74810223e+00 -1.35141307e+00
  2.84788854e+00 -5.64954153e-03]
iteration 78 accepted 8265.672905983329 [-1.14614316  7.85866882 -6.74923011 -1.35256255  2.84800596 -0.02054724]
iteration 79 accepted 8263.388580055922 [-1.13472101  7.85216278 -6.75136869 -1.34166362  2.86706992 -0.04599406]
iteration 80 accepted 8263.361278966493 [-1.13462823  7.85210735

iteration 3 accepted 7448.106186320481 [-0.60326623  6.6477604  -6.12268552  0.74864145  1.19577949  0.03873697]
iteration 4 accepted 7448.087290879178 [-0.60328139  6.64763267 -6.12274188  0.74877134  1.19487853  0.03929412]
iteration 5 converged 7448.08356392924 [-0.60328439  6.64760745 -6.12275301  0.74879699  1.19470058  0.03940417]
ITERATION 5 

iteration 0 accepted 7225.556656287925 [-0.50058371  6.4030052  -6.11746238  0.18309314  0.96493037  0.04816309]
iteration 1 accepted 7223.927044221702 [-0.52619153  6.46345354 -6.15737709  0.44087774  0.84106324  0.08211005]
iteration 2 accepted 7223.428196561615 [-0.53384468  6.47226217 -6.16993063  0.51270466  0.74563176  0.14059819]
iteration 3 accepted 7222.7440144951615 [-0.537425    6.47366618 -6.17321807  0.5321467   0.71487926  0.16163318]
iteration 4 accepted 7222.49565314282 [-0.5383758   6.47454322 -6.17589265  0.54660796  0.68987301  0.1780549 ]
iteration 5 accepted 7222.480742218054 [-0.53848773  6.47461628 -6.17621083  0.548

iteration 10 accepted 5063.247787855506 [-1.91810647e-01  6.25215138e+00 -6.28310263e+00  4.16748795e-03
  8.84431513e-03  7.77115947e-03]
iteration 11 accepted 5063.145128343503 [-1.92075188e-01  6.25212290e+00 -6.28310157e+00  4.18081564e-03
  8.85439715e-03  7.77698570e-03]
iteration 12 accepted 5063.1385529198205 [-1.92054483e-01  6.25212815e+00 -6.28310101e+00  4.18452253e-03
  8.85720560e-03  7.77748213e-03]
iteration 13 accepted 5063.132070393392 [-1.92106351e-01  6.25212329e+00 -6.28310065e+00  4.18826916e-03
  8.86005308e-03  7.77886123e-03]
iteration 14 accepted 5063.126693297114 [-1.92092391e-01  6.25212718e+00 -6.28310021e+00  4.19123178e-03
  8.86224112e-03  7.77924287e-03]
iteration 15 accepted 5063.121523407568 [-1.92134196e-01  6.25212330e+00 -6.28309992e+00  4.19424963e-03
  8.86448803e-03  7.78031600e-03]
iteration 16 converged 5063.116995898671 [-1.92124647e-01  6.25212627e+00 -6.28309956e+00  4.19667637e-03
  8.86623874e-03  7.78061725e-03]
ITERATION 14 

iteration 

In [11]:
dynamics.set_state(x0)
print(dynamics.get_state())
video = []
for i in range(us.shape[0]):
    dynamics.step(us[i])
    video.append(dynamics.sim.render(512, 512))

[0.         1.11694266 1.64382064 0.         0.         0.        ]


In [12]:
print(us.shape)

(187, 1)


In [13]:
make_video_fn(video)()